In [79]:
import numpy as np 
import pandas as pd
import math
import matplotlib.pyplot as plt 
import scipy.optimize
from scipy.stats import chi2, kstwobign, norm
from mpmath import mp

plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [80]:
n = 100
p = 1/10
columns = [i for i in range(0, 9+1)]
index = ["*.x*"]
data_df = pd.DataFrame(columns=columns, index = pd.Index(index))
data_df.loc["*.x*"] = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])
data_df

,0,1,2,3,4,5,6,7,8,9
*.x*,5,8,6,12,14,18,11,6,13,7


In [ ]:
p_even = [i/10 for i in range(1, 10+1)]
p_empirical = [data_df.loc["*.x*", 0]/n]
for i in range(1, 9+1):
    p_empirical.append(p_empirical[i-1] + data_df.loc["*.x*", i]/n)

# ПУНКТ Б

## проверяем критерий с помощью Теоремы о критерии Пирсона для сложной гипотезы
да

In [88]:
columns = ["(-inf, 1)", "[1, 2)", "[2, 3)", "[3, 4)", "[4, 5)", "[5, 6)", "[6, 7)", "[7, 8)", "[8, 9)", "[9, +inf)"]
index = pd.Index(["m_i"])
data_df = pd.DataFrame(columns=columns, index=index)
data_df.loc["m_i", :] = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])
data_df

,"(-inf, 1)","[1, 2)","[2, 3)","[3, 4)","[4, 5)","[5, 6)","[6, 7)","[7, 8)","[8, 9)","[9, +inf)"
m_i,5,8,6,12,14,18,11,6,13,7


### крутая библиотека mpmath сама считает за нас p_i, даже думать не надо!!!

In [89]:
def p_m(m: int, theta_1: float, theta_2: float) -> float:
    mp.dps = 50
    if m == 0:
        return mp.quad(lambda x: (1/(theta_2*math.sqrt(math.pi*2)))*mp.exp((-1/2)*(((x-theta_1)/(theta_2)))**2), [-1000, 1])
    elif 1 <= m <= 8:
        return mp.quad(lambda x: (1/(theta_2*math.sqrt(math.pi*2)))*mp.exp((-1/2)*(((x-theta_1)/(theta_2)))**2), [m, m+1])
    elif m == 9:
        return mp.quad(lambda x: (1/(theta_2*math.sqrt(math.pi*2)))*mp.exp((-1/2)*(((x-theta_1)/(theta_2)))**2), [9, 1000])
    else:
        raise ValueError("перезапустите бутстрап :D")


def normal_F(x: float, theta_1: float, theta_2: float) -> float:
    return mp.quad(lambda x: (1/(theta_2*math.sqrt(math.pi*2)))*mp.exp((-1/2)*(((x-theta_1)/(theta_2)))**2), [-1000, x])

### Функция правдоподобия для ОМПГ:

тут ф-ю правдободобия считаю (там в тетради формулы писал)

In [90]:
def L(m_i: tuple[float], theta_1: float, theta_2: float) -> float:
    ret = 1
    for m in range(0, 9+1):
        arg = p_m(m, theta_1, theta_2)
        if arg != 0:
            ret *= arg**m_i[m]
    return ret
def lnL(m_i: tuple[float], theta_1: float, theta_2: float) -> float:
    ret = 0
    for m in range(0, 9+1):
        arg = p_m(m, theta_1, theta_2)
        if arg <= 0.0:
            ret = float("+inf")
            break
        ret += m_i[m]*math.log(arg)
    return ret

def negative_lnL_m(theta: list[float]) -> float:
    return (-1)*lnL(np.array(data_df.loc["m_i", :]), theta[0], theta[1])

In [91]:
num_data = np.array([i for i in range(0, 9+1) for _ in range(data_df.loc["m_i", columns[i]])])
M = num_data.mean()
std = num_data.std()

print(f"M, сигма = {M}, {std}")

M, σ = 4.77, 2.505414137423193


In [92]:
res = scipy.optimize.minimize(negative_lnL_m, x0=[M, std])
print(f"(тета1, тета2) = {res.x}")

(theta_1, theta_2) = [5.28967723 2.67951979]


p_i, np_i считаем

In [94]:
p_i_df = pd.DataFrame(columns=columns, index=pd.Index(["p_i"]))
p_i_df.loc["p_i", :] = np.array([round(float(p_m(m, mu, sigma)), ndigits=4) for m in range(0, 9+1)])
p_i_df

,"(-inf, 1)","[1, 2)","[2, 3)","[3, 4)","[4, 5)","[5, 6)","[6, 7)","[7, 8)","[8, 9)","[9, +inf)"
p_i,0.0547,0.0551,0.0866,0.1187,0.1418,0.1476,0.1338,0.1058,0.0728,0.0831


In [95]:
n = 100
np_i_df = pd.DataFrame(columns=columns, index=pd.Index(["np_i"]))
np_i_df.loc["np_i", :] = np.array(p_i_df.loc["p_i", :]*n)
np_i_df

,"(-inf, 1)","[1, 2)","[2, 3)","[3, 4)","[4, 5)","[5, 6)","[6, 7)","[7, 8)","[8, 9)","[9, +inf)"
np_i,5.47,5.51,8.66,11.87,14.18,14.76,13.38,10.58,7.28,8.31


In [96]:
data_df.loc["p_i", :] = np.array([p_i_df.loc["p_i", :]])
data_df.loc["np_i", :] = np.array([np_i_df.loc["np_i", :]])
data_df

,"(-inf, 1)","[1, 2)","[2, 3)","[3, 4)","[4, 5)","[5, 6)","[6, 7)","[7, 8)","[8, 9)","[9, +inf)"
m_i,5,8,6,12,14,18,11,6,13,7
p_i,0.0547,0.0551,0.0866,0.1187,0.1418,0.1476,0.1338,0.1058,0.0728,0.0831
np_i,5.47,5.51,8.66,11.87,14.18,14.76,13.38,10.58,7.28,8.31


тут крит пирсона считаем (p-value находим)

In [97]:
delta = 0
for i in range(0, 9+1): delta += (n*data_df.loc["p_i", columns[i]] - data_df.loc["m_i", columns[i]])**2 / (n*p)
print(f"\u0303\u0394 = {delta}")

̃Δ = 8.511879999999998


In [98]:
sf = chi2.sf(delta, 10-1)
print(f"p-val = {sf}")

p-value = 0.4834936966175828


## теперь колмогоров

In [99]:
theta_1 = num_data.mean()
theta_2 = num_data.std()

In [100]:
n = 100
columns = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
index = pd.Index(["m_i", "p_i", "F_empirical_i"])
data_df = pd.DataFrame(columns=columns, index=index)
data_df.loc["m_i", :] = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])
data_df.loc["p_i", :] = np.array([m_i/n for m_i in data_df.loc["m_i", :]])
data_df.loc["F_empirical_i", :] = np.array([sum(list(data_df.loc["p_i", columns[:i+1]])) for i in range(0, 9+1)])
data_df

,0,1,2,3,4,5,6,7,8,9
m_i,5,8,6,12,14,18,11,6,13,7
p_i,0.05,0.08,0.06,0.12,0.14,0.18,0.11,0.06,0.13,0.07
F_empirical_i,0.05,0.13,0.19,0.31,0.45,0.63,0.74,0.8,0.93,1.0


In [ ]:
# Эмпирическая функция распределения
plt.step(columns, np.array(data_df.loc["F_empirical_i", :]), where='post')

# Функция распределения нормального распределения
_x = np.array(range(0, 200+1))/20
_y = np.array([normal_F(__x, theta_1, theta_2) for __x in _x])
plt.plot(_x, _y)

delta = 0
delta = max(abs(normal_F(0, theta_1, theta_2) - 0), abs(p_even[0] - p_empirical[0]))
for i in range(1, 9+1):
    delta = max(
        delta,
        abs(normal_F(i, theta_1, theta_2) - data_df.loc["F_empirical_i", i-1]),
        abs(normal_F(i, theta_1, theta_2) - data_df.loc["F_empirical_i", i])
    )
delta *= math.sqrt(n)
print(f"\u0303\u0394 = {delta}")

## Построение вариационного ряда с помощью параметрического бутстрапа :(

### 1) $\vec{x_n^*}$ $\rightarrow$ $\widetilde{\vec{\theta}}$ $\rightarrow$ $\Delta_1^*$ = $\sqrt{n}\displaystyle\sup\limits_{x \in \R}|\widetilde{F_{}^*}(x) - F(x, \widetilde{\vec{\theta_{}^*}})|$
### 2) $\vec{x_n^*}$ $\rightarrow$ $\widetilde{\vec{\theta}}$ $\rightarrow$ $\Delta_2^*$ = $\sqrt{n}\displaystyle\sup\limits_{x \in \R}|\widetilde{F_{}^*}(x) - F(x, \widetilde{\vec{\theta_{}^*}})|$
### ...
### N-1) $\vec{x_n^*}$ $\rightarrow$ $\widetilde{\vec{\theta}}$ $\rightarrow$ $\Delta_{N-1}^*$ = $\sqrt{n}\displaystyle\sup\limits_{x \in \R}|\widetilde{F_{}^*}(x) - F(x, \widetilde{\vec{\theta_{}^*}})|$
### N) $\vec{x_n^*}$ $\rightarrow$ $\widetilde{\vec{\theta}}$ $\rightarrow$ $\Delta_N^*$ = $\sqrt{n}\displaystyle\sup\limits_{x \in \R}|\widetilde{F_{}^*}(x) - F(x, \widetilde{\vec{\theta_{}^*}})|$
### Вариационный ряд:
### $\Delta_{(1)}^*$, $\Delta_{(2)}^*$, $\Delta_{(3)}^*$, ... $\Delta_{(N-1)}^*$, $\Delta_{(N)}^*$

In [ ]:
def kolmogorov_delta_bootstrap(data: np.array, theta_1: float, theta_2: float, dist) -> float:
    sample = dist.rvs(loc=theta_1, scale=theta_2, size=len(data))
    # Строим эмпирическую функцию распределения
    x = sorted(set(sample))
    sample_dict = dict()
    for value in sample:
        if value in sample_dict:
            sample_dict[value] += 1/len(sample)
        else:
            sample_dict[value] = 1/len(sample)
    F_empirical = [sample_dict[x[0]]]
    for i in range(1, len(x)):
        F_empirical.append(F_empirical[i-1] + sample_dict[x[i]])

    theta_new_1, theta_new_2 = dist.fit(sample)

    new_delta = 0
    new_delta = max(
        abs(dist.cdf(x[0], loc=theta_new_1, scale=theta_new_2) - 0),
        abs(F_empirical[0] - dist.cdf(x[0], loc=theta_new_1, scale=theta_new_2))
        )
    for i in range(1, len(x)):
        new_delta = max(
            new_delta,
            abs(dist.cdf(x[i], loc=theta_new_1, scale=theta_new_2) - F_empirical[i-1]),
            abs(dist.cdf(x[i], loc=theta_new_1, scale=theta_new_2) - F_empirical[i])
            )
    new_delta *= math.sqrt(n)

    return new_delta, theta_new_1, theta_new_2


def parametric_bootstrap_params(data, dist=norm, n_bootstrap=50000) -> tuple:
    original_theta_1, original_theta_2 = dist.fit(data)
    prev_theta_1, prev_theta_2 = original_theta_1, original_theta_2

    deltas = []
    params = [(original_theta_1, original_theta_2)]
    for _ in range(n_bootstrap):
        new_delta, theta_1, theta_2 = kolmogorov_delta_bootstrap(data, original_theta_1, original_theta_2, dist)
        prev_theta_1 = theta_1
        prev_theta_2 = theta_2
        params.append((prev_theta_1, prev_theta_2))
        deltas.append(new_delta)

    return sorted(deltas), params

real_data = [i for i in range(0, 9+1) for _ in range(data_df.loc["m_i", columns[i]])]
deltas, params = parametric_bootstrap_params(np.array(real_data, dtype=np.float32), dist=norm, n_bootstrap=50000)

### p-value = $\frac{k}{N}$, где k - кол-во элементов вариационного ряда $\{\Delta_{(n)}^*\}_1^N$ таких, что $\Delta_{(i)}^* \ge \widetilde{\Delta}$

In [712]:
count = 0
for _delta in deltas:
    if _delta >= delta:
        count += 1
print(f"p_value = {count/len(deltas)}")

p_value = 0.01354


p-val < a -> отвергаем